In [143]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

/kaggle/input/crime-prediction-b1-fall-inter-22-23/crime_test.csv
/kaggle/input/crime-prediction-b1-fall-inter-22-23/sample_submission.csv
/kaggle/input/crime-prediction-b1-fall-inter-22-23/crime_train.csv


In [144]:
#Load the training and test data
train_data = pd.read_csv('/kaggle/input/crime-prediction-b1-fall-inter-22-23/crime_train.csv')
test_data = pd.read_csv('/kaggle/input/crime-prediction-b1-fall-inter-22-23/crime_test.csv')

In [145]:
train_data.head()

,ID,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,...,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,ViolentCrimesPerPop
0,1,14985,2.56,16.55,34.42,22.54,10.13,0,0.0,35545,...,23.3,13.5,0,0,3.32,60.94,47.28,66.65,82.41,428.64
1,2,30843,2.83,15.45,35.12,18.14,4.70,0,0.0,32033,...,21.6,10.9,68,41,4.98,25.24,28.17,52.73,59.20,742.54
2,3,74991,2.52,10.48,20.43,9.11,20.68,73342,97.8,31177,...,23.6,11.9,0,0,6.51,13.69,34.49,60.88,66.97,303.72
3,4,45061,2.44,10.59,24.97,11.61,16.34,45061,100.0,39822,...,24.0,16.3,0,1,13.13,67.79,65.19,84.28,91.32,373.88
4,5,12863,2.45,12.02,22.51,10.49,18.46,0,0.0,23044,...,16.0,11.7,0,0,1.08,80.22,53.17,86.56,93.15,108.07


In [146]:
print(train_data.columns)

Index(['ID', 'population', 'householdsize', 'agePct12t21', 'agePct12t29',
       'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome',
       'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst',
       'pctWRetire', 'medFamInc', 'perCapInc', 'NumUnderPov', 'PctPopUnderPov',
       'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed',
       'PctEmploy', 'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu',
       'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr', 'FemalePctDiv',
       'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par',
       'PctYoungKids2Par', 'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom',
       'NumKidsBornNeverMar', 'PctKidsBornNeverMar', 'NumImmig',
       'PctImmigRecent', 'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10',
       'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8', 'PctRecImmig10',
       'PctSpeakEnglOnly', 'PctNotSpeakEnglWell', 'PctLargHouseFam',
       'PctLargHouseOccup', 'PersPerOccupHous

In [147]:
# Separate the target variable which is ViolentCrimesPerPop present in the last column from the rest features

X = train_data.iloc[:, 1:-1] # Exclude the first column and the last column
y = train_data.iloc[:, -1]
X

,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,...,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85
0,14985,2.56,16.55,34.42,22.54,10.13,0,0.0,35545,81.47,...,30.3,23.3,13.5,0,0,3.32,60.94,47.28,66.65,82.41
1,30843,2.83,15.45,35.12,18.14,4.70,0,0.0,32033,88.31,...,26.0,21.6,10.9,68,41,4.98,25.24,28.17,52.73,59.20
2,74991,2.52,10.48,20.43,9.11,20.68,73342,97.8,31177,68.06,...,26.7,23.6,11.9,0,0,6.51,13.69,34.49,60.88,66.97
3,45061,2.44,10.59,24.97,11.61,16.34,45061,100.0,39822,77.19,...,24.3,24.0,16.3,0,1,13.13,67.79,65.19,84.28,91.32
4,12863,2.45,12.02,22.51,10.49,18.46,0,0.0,23044,71.15,...,24.7,16.0,11.7,0,0,1.08,80.22,53.17,86.56,93.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,372242,2.58,12.88,27.02,12.96,10.63,372242,100.0,27095,72.09,...,30.7,25.3,12.8,431,303,19.75,45.34,48.53,79.89,89.05
1591,49389,2.56,11.17,22.59,10.65,18.16,49389,100.0,41745,74.97,...,26.6,22.4,15.0,3,3,6.16,68.75,69.46,92.04,95.25
1592,131591,2.62,12.40,28.85,13.61,11.28,131591,100.0,35103,78.21,...,29.8,25.1,11.8,220,14,27.53,39.13,45.87,80.95,84.68
1593,165121,2.36,16.31,32.25,18.75,13.77,165121,100.0,19923,72.35,...,26.5,20.4,13.6,354,130,1.98,69.57,47.86,77.77,87.92


In [148]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,...,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85
1548,11500,3.34,15.07,26.62,13.39,10.69,0,0.0,46566,87.77,...,24.9,22.5,13.8,10,0,6.04,75.48,59.11,79.21,92.25
1512,17906,2.42,11.58,20.95,10.00,21.35,17906,100.0,25380,64.78,...,29.5,25.3,12.7,5,0,10.66,25.03,48.40,70.72,78.23
744,10886,2.88,15.33,27.52,12.70,9.28,0,0.0,21946,77.74,...,29.7,17.6,11.6,0,0,7.16,77.13,57.62,82.01,94.19
462,22906,2.84,16.24,31.45,15.07,10.56,0,0.0,18858,73.98,...,27.7,20.4,16.9,26,0,1.10,52.64,41.83,71.16,74.08
1358,23146,2.53,13.35,23.90,11.11,16.62,0,0.0,19205,67.87,...,28.1,18.1,14.0,0,0,0.78,74.38,60.45,85.90,92.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,20908,2.53,14.96,25.79,11.87,16.32,0,0.0,17206,66.71,...,30.0,21.8,14.1,2,0,0.77,81.61,59.51,89.69,92.99
1294,25178,2.52,12.97,23.45,11.10,15.77,0,0.0,27325,75.42,...,23.4,17.0,12.8,10,0,1.56,79.93,58.48,84.11,93.90
860,17140,2.88,14.80,27.74,12.50,7.77,0,0.0,32098,84.93,...,22.3,17.7,13.6,0,0,2.44,74.21,50.21,84.89,95.31
1459,18609,2.84,16.38,30.27,14.52,5.80,18609,100.0,37050,89.29,...,24.3,19.2,10.9,0,0,2.05,63.49,55.61,82.89,90.25


In [149]:
# Calculate the correlation matrix to get the feature importance
correlation_matrix = X_train.corr()

# Sort the features based on their correlation with the target variable
correlation_with_target = abs(correlation_matrix.iloc[:, -1]).sort_values(ascending=False)

In [150]:
# Select the top k features
k = 82
selected_features = correlation_with_target.index[:k]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]
X_test_selected

,PctSameState85,PctBornSameState,PctSameCity85,PctSameHouse85,MedYrHousBuilt,pctWSocSec,PctImmigRec8,PctOccupManu,PctImmigRec5,PctNotHSGrad,...,PctNotSpeakEnglWell,TotalPctDiv,PctKidsBornNeverMar,numbUrban,population,NumImmig,NumInShelters,MalePctDivorce,PersPerRentOccHous,PctWorkMom
917,90.72,76.71,86.36,55.06,1939,33.29,40.04,26.73,28.82,41.57,...,5.07,14.07,8.86,78380,78380,3362,172,12.30,2.30,64.73
1120,92.46,78.18,80.26,48.81,1954,32.69,25.48,24.75,19.75,23.50,...,0.07,12.72,2.51,0,13584,157,0,10.14,2.04,75.47
1276,92.75,80.78,85.75,57.46,1966,30.69,34.43,21.56,5.46,38.36,...,0.45,13.48,7.69,0,18601,183,2,10.40,2.50,71.93
361,81.44,55.31,74.18,48.64,1963,21.63,34.98,10.55,19.45,15.78,...,0.98,11.11,0.96,43929,43929,1095,32,9.58,2.33,67.78
1484,75.16,47.98,67.71,36.30,1980,17.02,15.34,5.87,6.08,9.74,...,0.35,10.41,1.68,30108,30108,691,0,8.48,2.18,73.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,82.42,44.14,57.07,37.31,1981,7.86,39.52,8.12,22.63,10.15,...,3.44,11.77,1.59,82169,82169,9387,0,9.05,2.43,72.17
701,92.78,74.92,88.25,54.28,1948,25.23,13.42,14.31,8.37,20.07,...,0.55,10.59,2.63,739,14997,514,19,9.25,2.30,70.49
1585,93.72,65.68,81.92,63.83,1958,34.03,17.28,10.22,12.22,20.69,...,0.73,8.00,2.05,34185,34185,1956,0,7.22,1.94,76.29
350,92.01,57.76,88.34,59.70,1978,30.76,27.05,17.66,19.61,58.41,...,22.83,7.60,3.07,32921,32921,8583,34,5.04,3.87,50.10


In [151]:
z = test_data.iloc[:, 1:]  # Exclude the first column
z

,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,...,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85
0,11874,2.11,10.54,30.87,14.08,8.16,11874,100.0,30141,84.74,...,22.9,19.2,11.4,2,1,7.02,51.78,39.89,75.63,83.39
1,14143,2.68,21.01,33.35,21.95,14.55,0,0.0,23886,72.48,...,30.5,21.0,14.9,0,0,2.35,75.91,51.79,72.20,88.40
2,34882,2.32,12.56,21.79,11.29,19.51,34881,100.0,21053,64.58,...,26.5,16.9,11.8,0,9,1.61,74.43,62.60,86.35,90.85
3,29885,3.53,20.10,34.33,18.31,8.18,29885,100.0,18956,78.48,...,26.2,19.8,13.8,0,0,18.46,68.90,52.42,83.01,90.76
4,935933,2.86,15.89,30.35,14.98,9.50,935927,100.0,23584,79.02,...,25.3,20.9,12.7,698,44,9.35,69.25,51.62,83.78,90.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,11096,2.40,26.67,47.49,36.51,11.11,0,0.0,22265,80.70,...,35.1,17.8,11.9,0,0,6.50,42.07,36.22,47.90,70.05
395,11843,2.45,12.77,24.08,11.09,15.82,0,0.0,21321,69.65,...,25.9,18.5,15.5,12,0,1.08,65.35,64.44,84.86,89.83
396,33892,2.80,16.81,28.88,13.91,12.35,33892,100.0,19391,67.09,...,25.0,15.9,12.8,0,5,11.56,60.92,66.23,92.98,93.49
397,29060,2.76,15.46,27.15,12.78,9.38,29060,100.0,33118,81.80,...,23.0,15.3,12.4,2,0,2.43,66.47,57.27,77.55,92.13


In [152]:
# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

In [153]:
model = LinearRegression()

model.fit(X_train_scaled, y_train)

LinearRegression()

In [154]:
# Make predictions on the test set
z_test_data = z[selected_features]
z_test_data_scaled = scaler.transform(z_test_data)
predictions = model.predict(z_test_data_scaled)

In [158]:
# Create a submission file
submission = pd.DataFrame({'ID': test_data.index, 'Prediction': predictions})
submission['ID'] = submission.index + 1
submission.to_csv('submission.csv', index=False)

In [156]:
submission

,ID,Prediction
0,1,737.25
1,2,389.00
2,3,600.25
3,4,665.50
4,5,1103.50
...,...,...
394,395,426.00
395,396,-21.50
396,397,1381.25
397,398,404.50


In [160]:
# Evaluate the model
mse = mean_squared_error(y_test, predictions[:319])
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 789.728667463376


In [157]:
s = pd.read_csv('/kaggle/input/crime-prediction-b1-fall-inter-22-23/sample_submission.csv')
s

,ID,Prediction
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
394,395,1
395,396,1
396,397,1
397,398,1
